## (1) Getting started + Julia basics

* Install Julia. Add the following packages: JuMP, IJulia, Clp, SCS, ECOS, NamedArrays.
* Create, name, and run an IJulia notebook. Save it as a PDF.

### Hopefully all the above tasks went smoothly!

## (2) LP Basics

- Farmer Jane owns 45 acres of land. She is going to plant each with wheat or corn. Each acre planted with wheat yields \\$200 proﬁt; each with corn yields \\$300 proﬁt. The labor and fertilizer used for each acre are given in the table below. One hundred workers and 120 tons of fertilizer are available. How should Jane plant her crops to maximize proﬁt? 

    - Write down the full mathematical model. 
     
    - Solve the problem using the graphical method. 
     
    - Implement and solve the problem in Julia.
     
|            | Wheat     | Corn      |
|-----------:|-----------|-----------|
| Labor      | 3 workers | 2 workers |
| Fertilizer | 2 tons    | 4 tons    |


 ### Solutions  

 * Decision variables: $x_w$ = acres of wheat planted. $x_c$ = acres of corn planted. Both nonnegative
 * Objective: maximize profit: $200x_w + 300x_c$
 * Constraints: Labor ($3x_w + 2x_c \leq 100$), Fertilizer ($2x_w + 4x_c \leq 120$), and Land ($x_w + x_c \leq 45$)
 
 Full math model:
 
 $\begin{align}
 \max_x \ & 200x_w + 300x_c\\
 \text{s.t.} \ & 3x_w + 2x_c \leq 100\\
 & 2x_w + 4x_c \leq 120\\
 & x_w + x_c \leq 45\\
 & x_w, x_c \geq 0.
 \end{align}$

Solved with graphical method:

![](week1_graphical.jpg)

Solution is at $x_w = x_c = 20$ acres. Profit is \\$1,000.

And in Julia:

In [1]:
using JuMP, Clp

m = Model(Clp.Optimizer)

@variable(m, xw >= 0)
@variable(m, xc >= 0)

@objective(m, Max, 200xw + 300xc)

@constraints(m,
    begin
    3xw + 2xc <= 100 # labor
    2xw + 4xc <= 120 # fertilizer
    xw + xc <= 45 # land
    end
)

optimize!(m)

println("Jane should plant ", value(xw), " acres of wheat and ", value(xc), " acres of corn.")

Jane should plant 19.99999999999999 acres of wheat and 20.000000000000007 acres of corn.
Coin0506I Presolve 3 (0) rows, 2 (0) columns and 6 (0) elements
Clp0006I 0  Obj 0 Dual inf 500 (2)
Clp0006I 3  Obj 10000
Clp0000I Optimal - objective value 10000
Clp0032I Optimal objective 10000 - 3 iterations time 0.032



* For each of the three Top Brass implementations (Top Brass, Top Brass Modular, Top Brass Compact), re-implement the model to add a new trophy type: karate trophies. Karate trophies require 3 board feet of wood, 1 plaque, and 1 brass karate figure on top. Karate trophies are sold for \\$10 each. There are 750 brass karate figures available. Comment on how easy or hard it is to add a new trophy type in each case.

### Solutions

In [2]:
### Original Top Brass ###
using JuMP

m = Model()

@variable(m, ft >= 0)
@variable(m, st >= 0)
@variable(m, kt >= 0)

# objective is to maximize profit
# format is (<model name>, <Max or Min>, <algebraic function>)
@objective(m, Max, 12*ft + 9*st + 10*kt)

# constraint on the wood available
# format is (<model name>, <constraint name>, <algebraic constraint>)
@constraint(m, wood_con, 4ft + 2st + 3kt <= 4800)

#constraint on the plaques available
@constraint(m, plaque_con, ft + st + kt <= 1750)

# constraints on brass footballs, soccerballs available
@constraint(m, brass_football_con, ft <= 1000)
@constraint(m, brass_soccerball_con, st <= 1500)
@constraint(m, karate_fig_con, kt <= 750)
; 

    #Not too hard, just had to modify each component of the model slightly
    
### Modular Top Brass ###

# Data first 
trophy_types = [:football, :soccer, :karate] 

wood_req = Dict(:football => 4, :soccer => 2, :karate => 3) # how much wood each trophy type will use

plaque_req = Dict(:football => 1, :soccer => 1, :karate => 1) # how many plaques each trophy type will use

profit = Dict( :football => 12, :soccer => 9, :karate => 10) # profit produced by each trophy type

# we are told the amount of each resource we have available
wood_avail = 4800
plaques_avail = 1750
football_avail = 1000
soccer_avail = 1500
karate_avail = 740;

# Now model:

m = Model()

# trophy variable object is now a Dictionary indexed over trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# maximize profit by summing (profit/trophy * # trophies) for each type
@objective(m, Max, sum(profit[i] * trophy[i] for i in trophy_types) )   

@constraint(m, sum(wood_req[i] * trophy[i] for i in trophy_types) <= wood_avail) # use only available wood
@constraint(m, sum(plaque_req[i] * trophy[i] for i in trophy_types) <= plaques_avail) # use only available plaques
@constraint(m, trophy[:football] <= football_avail)  # use only available brass footballs
@constraint(m, trophy[:soccer] <= soccer_avail)  # use only available brass soccer balls
@constraint(m, trophy[:karate] <= karate_avail)  # use only available brass soccer balls

    #Similar to old Top Brass. Very little to modify within the model.
    
### Compact Top Brass ###

# Data:
trophy_types = [:football, :soccer, :karate] # these are the possible trophy types

resources = [:wood, :plaques, :brass_football, :brass_soccer, :brass_karate] # what goes into each trophy

profit = Dict( zip(trophy_types, [12,9, 10] ) ) # profit produced by each trophy type

resource_avail = Dict( zip(resources, [4800, 1750, 1000, 1500, 750] ) ); # amount of each type of resource available

using NamedArrays

trophy_resource_matrix = [4 1 1 0 0
                            2 1 0 1 0
                            3 1 0 0 1]

trophy_resource_NA = NamedArray(trophy_resource_matrix, (trophy_types, resources), ("type","resource"))
;

# Model:

m = Model()

# trophy variable object is now a Dictionary indexed over trophy types (elements are variables)
@variable(m, trophy[trophy_types] >= 0)

# use an expression object to calculate the total profit
@expression(m, tot_profit, sum(profit[i] * trophy[i] for i in trophy_types) )

# our trophy/resource NamedArray allows us to create a Dictionary of constraints.
# indices are resources, and elements are constraint objects.
@constraint(m, constr[i in resources], sum(trophy_resource_NA[t, i] * trophy[t] for t in trophy_types) <= resource_avail[i] )

# our objective is to maximize the total profit
@objective(m, Max, tot_profit)

    # Didn't have to change the model AT ALL! Just some slight adjustments to the data.
;



* A very famous optimization problem is the so-called “Stigler Diet Problem,” which was created in 1945 by American economist and Nobel laureate George Stigler.  Stigler published a paper on his study of the optimal diet, which minimizes total annual cost while meeting the recommended daily allowance (RDA)  of  several  nutrients.   In  his  paper,  Stigler  created  a  table  of  77  different  foods  (although  if you  look  at  the  data,  you’ll  note  that  one  of  the  foods  is  “Plate”...not  sure  about  that  one)  and their nutrient content for 9 nutrients:  calories, protein, calcium, iron, vitamin A, thiamine, riboflavin, niacin,  and ascorbic acid.  To make the calculations easier,  Stigler normalized the data so that the nutrients shown are the content for \\$1-worth of the given food.  \\$1 could buy you a lot more in 1945! You can find a sample of the first few rows and columns of the table below. Stigler published this paper before linear programming algorithms had been invented, but Stigler was a very intelligent man and used “trial and error, mathematical insight, and agility,” to arrive at a diet costing only \\$39.93 per year.  Stigler stated there was no reason to believe he had found the cheapest diet, and other combinations could prove more cost-effective.
    * Formulate  Stigler's  diet  problem  as  a  linear  program  and  solve  it  to  find  the  actual  optimal solution.   Stigler's  original  data  is  provided  as "stigler.csv" on Canvas. Use the code  snippet posted on Canvas as "stigler.jl" to import the data into arrays that will be much easier to work with. How does your cheapest  diet  compare  in  annual  cost  to  Stigler's?   For the purposes of this exercise, a year is 365.25 days.
    * Now suppose you wanted a diet consisting only of foods you like to eat.  Modify the stigler.csv file  by  deleting  any  rows  you  want,  leaving  only  your  favorite  foods  as  options  for  your  diet. Solve the diet problem again and compare your total annual cost to the cost you got in the previous problem.  What foods make up your optimal diet now? 

Table from the diet problem:

|                          | kCalories | Protein (g) | Calcium (g) | Iron (mg) | ...      |
|--------------------------|-----------|-------------|-------------|-----------|----------|
| Wheat Flour (Enriched)   | 44.7      | 1411        | 2           | 365       | ...      |
| Macaroni                 | 11.6      | 418         | 0.7         | 54        | ...      |
| Wheat Cereal (Enriched)  | 11.8      | 377         | 14.4        | 175       | ...      |
| $\vdots$                 | $\vdots$  | $\vdots$    | $\vdots$    | $\vdots$  | $\ddots$ |


### Solutions

In [4]:
using DataFrames, CSV, NamedArrays

#Note that the new syntax differs a bit from what is in stigler.jl. This is the non-deprecated version.
df = CSV.read("stigler.csv",DataFrame, delim =',');
# the names of the DataFrame (header) are the nutrients
nutrients = propertynames(df)[2:end]

# create a list of foods from the diet array
foods = convert(Array,df[2:end,1]) # turn dataframe into Array
# create a dictionary of the min requirement of each nutrient
min_daily_req = Dict(zip(nutrients,df[1,2:end]))

# create a NamedArray that specifies how much of each nutrient each food provides
using NamedArrays
food_nutrient_matrix = Matrix(df[2:end,2:end]) # turn dataframe into Array
# rows are foods, columns are nutrients
food_nutrient_array = NamedArray(food_nutrient_matrix, (foods, nutrients), ("foods","nutrients"))

using JuMP,  Clp
m = Model(Clp.Optimizer) # create model

@variable(m, x[foods] >= 0)

@objective(m, Min, sum(x))

@constraint(m, meet_req[n in nutrients], sum(food_nutrient_array[i,n] * x[i] for i in foods) >= min_daily_req[n])

optimize!(m)

println("Annual cost of this diet \$", 365.25objective_value(m),2)
println("How much of each food should Stigler eat every day? ")
for i in foods
    if value(x[i]) > 10e-5
        println(i, ": ", value(x[i]))
    end
end

Annual cost of this diet $39.688897115017942
How much of each food should Stigler eat every day? 
Wheat Flour (Enriched): 0.02951906167648827
Liver (Beef): 0.0018925572907052643
Cabbage: 0.011214435246144865
Spinach: 0.005007660466725203
Navy Beans, Dried: 0.061028563526693246
Coin0506I Presolve 9 (0) rows, 76 (-1) columns and 569 (-1) elements
Clp0006I 0  Obj 0 Primal inf 5.1310537 (9)
Clp0006I 6  Obj 0.10866228
Clp0000I Optimal - objective value 0.10866228
Coin0511I After Postsolve, objective 0.10866228, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0.1086622782 - 6 iterations time 0.002, Presolve 0.00




* Convert the following LP into standard form:

\begin{align*}
\underset{x_1,x_2,x_3,x_4}{\min} \ & 3x_1 - x_2 & \\
\text{s.t.} \ & -x_1 + 6x_2 - x_3 + x_4 \geq  -3 &\\
& 7x_2 + x_4 =  5 &\\
& x_3 + x_4 \leq 5 &\\
& -1 \leq x_2 \leq 5, \ -1 \leq x_3 \leq 5, \ -2 \leq x_4 \leq 2 &
\end{align*}

### Solutions
\begin{align*}
\underset{u,v,w,r,s}{-\max} \ & -3u + 3v + w -1 & \\
\text{s.t.} \ & u - v - 6w + r -s \leq -4&\\
& 7w + s \leq 14 &\\
& -7w - s \leq -14 &\\
& r + s \leq 8 &\\
& r \leq 6\\
& w \leq 6\\
& s \leq 4\\
& u, v, w, r, s \geq 0
\end{align*}

Where $x_1 = u - v$, $x_2 = w-1$, $x_3 = r-1$, and $x_4 = s - 2$.


## LP Solution Behavior 

* After modeling your LP and solving it, the solver unexpectedly returns “infeasible.” Which of the following modeling errors could have caused this?

    (a) Incorrect objective function
    
    (b) Forgot to include a constraint
    
    (c) Added an extra constraint by mistake
    
    (d) Any of (a), (b), or (c) could have caused it
    
    (e) Either (a) or (b) could have caused it
 

__(c) Is the correct answer. Feasible region is empty, so it is overconstrained.__
 
*  After modeling your LP and solving it, the solver unexpectedly returns “unbounded.” Which of the following modeling errors could have caused this?

    (a) Incorrect objective function
    
    (b) Forgot to include a constraint
    
    (c) Added an extra constraint by mistake
    
    (d) Any of (a), (b), or (c) could have caused it
    
    (e) Either (a) or (b) could have caused it
    
__(e) is the correct answer. The feasible region could be unbounded in the direction we are optimizing, indicating that we forgot part of the polyhedron (a constraint -- (b)). The objective could also be going the "wrong way" (maybe we forgot a negative sign) and be allowed to increase/decrease to infinity (objective wrong -- (a)).__

*  Suppose you have a constrained minimization problem. You then add an extra linear constraint to the problem. What might the optimal objective value do?

    (a) increase
    
    (b) decrease
    
    (c) stay the same
    
    (d) Any of (a), (b), or (c) could have caused it
    
    (e) Either (a) or (c) could have caused it
   
 __(e) is the correct answer. The new constraint can only REMOVE choices, not give new choices. Therefore, if it cuts off the previously optimal point, the objective will get worse (increase). Or, it could not affect the optimal point (e.g., the new constraint is satisfied by every point in the feasible set) so the objective doesn't change. It can never get better (decrease) by adding a new constraint.__

In [5]:
using JuMP, Clp
m = Model(Clp.Optimizer)
@variable(m, u1 >= 0)
@variable(m, v1 >= 0)
@variable(m, 0 <= v2 <= 6)
@variable(m, 0 <= v3 <= 6)
@variable(m, 0 <= v4 <= 4)
@objective(m, Max, v2 -1 -3*u1 + 3*v1)
@constraint(m, u1 - v1 -6*v2 + v3 - v4 <= -4)
@constraint(m, 7*v2 + v4 <= 12)
@constraint(m, -7*v2 - v4 <= -12)
@constraint(m, v3 + v4 <= 8)
optimize!(m)
println(m)
println("x1 = ", value(u1-v1))
println("x2 = ", value(v2-1))
println("x3 = ", value(v3-1))
println("x4 = ", value(v4-2))
println("objective = ", -objective_value(m))


m = Model(Clp.Optimizer)
@variable(m,x[1:4])

@objective(m, Min, 3x[1]-x[2])

@constraint(m, -1x[1]+6x[2]+-1x[3]+x[4]>=-3)
@constraint(m, 7x[2]+x[4] == 5)
@constraint(m, x[3]+x[4]  <=5)
@constraint(m, -1 <= x[2] <= 5)
@constraint(m, -1 <= x[3] <= 5)
@constraint(m, -2 <= x[4] <= 2)

optimize!(m)

Max v2 - 3 u1 + 3 v1 - 1
Subject to
 u1 - v1 - 6 v2 + v3 - v4 <= -4.0
 7 v2 + v4 <= 12.0
 -7 v2 - v4 <= -12.0
 v3 + v4 <= 8.0
 u1 >= 0.0
 v1 >= 0.0
 v2 >= 0.0
 v3 >= 0.0
 v4 >= 0.0
 v2 <= 6.0
 v3 <= 6.0
 v4 <= 4.0

x1 = -9.78986545550759e10
x2 = -1.0
x3 = -1.0
x4 = -2.0
objective = -2.9369596366522766e11
Coin0508I Presolve thinks problem is unbounded
Clp3003W Analysis indicates model infeasible or unbounded
Clp0006I 0  Obj -1 Primal inf 7.717083 (2) Dual inf 6.1760409 (2)
Clp0006I 1  Obj 5.8739193e+10
Clp0006I 1  Obj 2.9369596e+11
Clp0002I Dual infeasible - objective value 2.9369596e+11
Clp0032I DualInfeasible objective 2.936959637e+11 - 1 iterations time 0.012
Coin0508I Presolve thinks problem is unbounded
Clp3003W Analysis indicates model infeasible or unbounded
Clp0006I 0  Obj 0 Primal inf 1.8898223 (1) Dual inf 0.075861727 (2) w.o. free dual inf (0)
Clp0006I 2  Obj 21.142857 Dual inf 7.2082282 (1)
Clp0006I 2  Obj -9.4587995e+10 Primal inf 1.0831456e+11 (4) Dual inf 4.3325825e+11 (3